In [3]:
# Minicons Installation
# Introduction can be found https://kanishka.xyz/post/minicons-running-large-scale-behavioral-analyses-on-transformer-lms/
# Tutorial and code can be found https://github.com/kanishkamisra/minicons/blob/master/examples/surprisals.md
#!pip install minicons

from minicons import scorer
import pandas as pd
import numpy as np
import json
import csv
import re
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.formula.api as smf
import torch
from torch.utils.data import DataLoader
from transformers import GPT2LMHeadModel, GPT2TokenizerFast
from transformers import AutoTokenizer, AutoModelForCausalLM

In [4]:
model_name = "ltg/gpt-bert-babylm-base"
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code = True)
model = AutoModelForCausalLM.from_pretrained(model_name, trust_remote_code = True)

lm_scorer = scorer.IncrementalLMScorer(model = model, tokenizer = tokenizer)

/home/stellacy/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `2.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/stellacy/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `2.5` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/stellacy/.local/lib/python3.9/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This sho

In [5]:
def calculate_surprisal(sentence):
    
    # encode input text
    inputs = tokenizer(sentence, return_tensors = "pt")
    
    # forward pass to get logits
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs[0]
    
    # calculate log-probabilities and surprisal
    log_probs = torch.nn.functional.log_softmax(logits, dim = -1)
    
    # get token IDs and compute surprisal for each token in the sentence
    input_ids = inputs.input_ids[0]
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    
    token_surprisals = []
    for token, token_id in zip(tokens, input_ids):
        if token in tokenizer.all_special_tokens:
            continue  # skip special tokens
        # get the log-probability of the token
        log_prob = log_probs[0, tokens.index(token), token_id]  # use token index and id to find the log prob
        score = -log_prob.item()  # convert to scalar
        token_surprisals.append((token, score))
        
    # expand tokens that contain multiple words
    expanded = []
    for token, score in token_surprisals:
    #for token, score in filtered:
        token = token.strip('Ġ') # remove space marker
        if token.count('Ġ') > 0:
            # multiple words inside
            words = token.split('Ġ')
            words = [word for word in words if word]  # remove empty strings
            for i, word in enumerate(words):
                expanded_token = word
                expanded.append((expanded_token, score / len(words)))  # split surprisal equally
        else:
            expanded.append((token, score))    

    # use regex to split into words and punctuation
    words = re.findall(r'\w+|[^\w\s]', sentence)
    results = []

    token_pointer = 0

    for word in words:
        accumulated = ''
        word_surprisal = 0.0

        while token_pointer < len(expanded):
            token, surprisal = expanded[token_pointer]
            accumulated += token
            word_surprisal += surprisal
            token_pointer += 1

            if accumulated == word:
                results.append((word, word_surprisal))
                break
        else:
            results.append((word, word_surprisal))

    return results

sentence = "I know that your friend gave a baguette to Mary last weekend."
surprisal_values = calculate_surprisal(sentence)
surprisal_values

[('I', 7.033481597900391),
 ('know', 9.674674034118652),
 ('that', 5.322581768035889),
 ('your', 7.956361770629883),
 ('friend', 9.01833724975586),
 ('gave', 11.33497428894043),
 ('a', 6.616059303283691),
 ('baguette', 31.180787563323975),
 ('to', 7.858166694641113),
 ('Mary', 7.558891296386719),
 ('last', 9.912156105041504),
 ('weekend', 11.895618438720703),
 ('.', 9.712087631225586)]

In [6]:
def calculate_sentence_surprisal(word_surprisals):
    '''Returns total surprisal and average surprisal per word.'''
    
    scores = [score for word, score in word_surprisals]
    total = sum(scores)
    avg = total / len(scores)
    return total, avg

"""
def sum_region_surprisal(row, region_list):
    '''Sums surprisals of a specified sentence region, extracting the relevant part from sentence_surprisals using index information.'''

    sentence_surprisals = row['surprisals']
    
    # retrieve all sentence column(from 'prefix' till 'end')
    all_columns = row.index.tolist()  # get the list of column names
    prefix_index = all_columns.index('prefix')
    eos_index = all_columns.index('end')
    
    # reconstruct the full sentence from 'prefix' to 'end' (exclude NaN values)
    full_sentence = ' '.join([str(row[col]) if pd.notna(row.get(col)) else '' for col in all_columns[prefix_index:eos_index + 1]])
    
    # use regex to split the full sentence into words & punctuation
    sentence_units = re.findall(r'\w+|[^\w\s]', full_sentence)
    
    # get the start and end indices for the region_list within the full sentence
    first_region_column = region_list[0]
    first_region_start_index = all_columns.index(first_region_column)
    
    last_region_column = region_list[-1]
    last_region_end_index = all_columns.index(last_region_column)
    
    # extract the relevant slice of sentence_surprisals that corresponds to the region_list
    # calculate the starting and ending index for the region_list part
    start_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:first_region_start_index]
        if pd.notna(row[col]) 
    ])
    end_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:last_region_end_index + 1]
        if pd.notna(row[col]) 
    ])

    # extract the slice of sentence_surprisals corresponding to the region_list part
    relevant_surprisals = sentence_surprisals[start_index:end_index]
    region_surprisal = sum([score for token, score in relevant_surprisals])
    
    return region_surprisal
"""

def sum_region_surprisal(row, region_list, priority_region = None, normalize = False):
    '''Sums surprisals of a specified sentence region with optional priority region handling.'''

    sentence_surprisals = row['surprisals']
    
    # retrieve all sentence column(from 'prefix' till 'end')
    all_columns = row.index.tolist()  # get the list of column names
    prefix_index = all_columns.index('prefix')
    eos_index = all_columns.index('end')
    
    # reconstruct the full sentence from 'prefix' to 'end' (exclude NaN values)
    full_sentence = ' '.join([str(row[col]) if pd.notna(row.get(col)) else '' for col in all_columns[prefix_index:eos_index + 1]])
    
    # use regex to split the full sentence into words & punctuation
    sentence_units = re.findall(r'\w+|[^\w\s]', full_sentence)
    
    if priority_region:
        # check if priority_region has non-zero surprisal
        if pd.notna(row[priority_region]):
            # extract text for the priority_region
            priority_region_index = all_columns.index(priority_region)
            
            start_index = sum([
                len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:priority_region_index]
                if pd.notna(row[col]) 
            ])
            end_index = sum([
                len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:priority_region_index + 1]
                if pd.notna(row[col]) 
            ])
            
            # calculate the surprisal for the priority region
            relevant_surprisals = sentence_surprisals[start_index:end_index]
            priority_surprisal = sum([score for token, score in relevant_surprisals])
    
            # return priority region's surprisal score if it is non-zero
            if priority_surprisal != 0:
                return priority_surprisal
    
    # if priority region is not provided or its surprisal is zero, calculate sum of region_list surprisals

    # get the start and end indices for the region_list within the full sentence
    first_region_column = region_list[0]
    first_region_start_index = all_columns.index(first_region_column)
    
    last_region_column = region_list[-1]
    last_region_end_index = all_columns.index(last_region_column)
    
    # extract the relevant slice of sentence_surprisals that corresponds to the region_list
    # calculate the starting and ending index for the region_list part
    start_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:first_region_start_index]
        if pd.notna(row[col]) 
    ])
    end_index = sum([
        len(re.findall(r'\w+|[^\w\s]', str(row[col]))) for col in all_columns[prefix_index:last_region_end_index + 1]
        if pd.notna(row[col]) 
    ])

    # extract the slice of sentence_surprisals corresponding to the region_list part
    relevant_surprisals = sentence_surprisals[start_index:end_index]
    region_surprisal = sum([score for token, score in relevant_surprisals])
    
    if normalize == False:
        return region_surprisal
    elif normalize == True:
        return region_surprisal / len(sentence_units)

In [7]:
def split_ends(ends):
    '''Splits off sentence-final punctuation.'''
    
    match = re.match(r'^(.*?)([.!?])$', ends.strip())
    if match:
        return pd.Series([match.group(1), match.group(2)])
    else:
        return pd.Series([ends, ''])  # no end punctuation
    
def encode_wh_licensor(df):
    '''
    Adds a numeric column to the DataFrame:
    - 1 if licensor is a wh-licensor
    - -1 otherwise (licensor is "that" or absent)
    '''
    df['wh_numeric'] = df['filler'].apply(lambda x: 1 if x > 0 else -1)
    return df
    
def merge_sentence(row, syntactic_parts):
    '''Merges sentence columns back into full sentence.'''
    
    parts = [row[part] for part in syntactic_parts]         
    # filter out NaN or empty parts
    non_empty_parts = [str(part).strip() for part in parts if pd.notna(part) and str(part).strip() != '']
    # join with spaces and add end punctuation
    sentence = ' '.join(non_empty_parts) + (row['end'] if pd.notna(row['end']) else '')
    return sentence

def assign_grammaticality(row):
    ''' Determines grammaticality based on licensor and gap values.'''
    if row['filler'] > 0 and row['gap'] == 1:
        return 'gram+'
        #return 1
    elif row['filler'] == 0 and row['gap'] == 0:
        return 'gram+'
        #return 1
    elif row['filler'] > 0 and row['gap'] == 0:
        return 'gram-'
        #return 0
    elif row['filler'] == 0 and row['gap'] == 1:
        return 'gram-'
        #return 0

#### Statistical Analysis: Mixed-Effects Linear Regression Model

In [8]:
from IPython.display import display
import warnings
from statsmodels.tools.sm_exceptions import ConvergenceWarning

def print_summary(result):
    '''
    Prints the descriptive header from a statsmodels MixedLMResults summary,
    and displays both the fixed-effects and random-effects components as pandas DataFrames.
    '''
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", ConvergenceWarning)
        
        summary_str = str(result.summary())
        lines = summary_str.split('\n')

        # PART 1: HEADER
        print("\n=== MODEL SUMMARY ===")
        for line in lines:
            if 'Coef.' in line:
                break  # stop when reaching the coefficient table
            pairs = re.findall(r'(\S[^:]*):\s+([^\s][^:]*?)(?=\s{2,}|$)', line)
            for key, value in pairs:
                print(f"{key.strip()}: {value.strip()}")

        # PART 2a: FIXED EFFECTS TABLE
        fixed_table = []
        in_fixed_table = False
        random_start_idx = None

        for i, line in enumerate(lines):
            if 'Coef.' in line:
                in_fixed_table = True
                continue
            if in_fixed_table:
                if not line.strip() or set(line.strip()) in [{'='}, {'-'}]:
                    continue  # skip empty/separator lines
                parts = line.strip().split()
                if len(parts) >= 7:
                    param = parts[0]
                    row = parts[1:7]
                    fixed_table.append([param] + row)
                else:
                    random_start_idx = i
                    break

        if fixed_table:
            df_fixed = pd.DataFrame(fixed_table, columns = [
                'Parameter', 'Coef.', 'Std.Err.', 'z', 'P>|z|', '[0.025', '0.975]'
            ])
            print("\n=== FIXED EFFECTS COEFFICIENTS ===")
            display(df_fixed)
        else:
            print("\nNo fixed-effects data found.")

        # PART 2b: RANDOM EFFECTS / VARIANCE COMPONENTS TABLE
        random_table = []
        if random_start_idx:
            for line in lines[random_start_idx:]:
                if not line.strip() or set(line.strip()) in [{'='}, {'-'}]:
                    continue
                parts = line.strip().split()
                # Look for last token ending in Var or Cov
                varcov_idx = None
                for j in reversed(range(len(parts))):
                    if parts[j].endswith('Var') or parts[j].endswith('Cov'):
                        varcov_idx = j
                        break
                if varcov_idx is not None:
                    param = ' '.join(parts[:varcov_idx + 1])
                    numeric_parts = parts[varcov_idx + 1:]
                    if len(numeric_parts) == 2:
                        val1, val2 = numeric_parts
                    elif len(numeric_parts) == 1:
                        val1, val2 = numeric_parts[0], ''
                    else:
                        val1, val2 = '', ''
                    random_table.append([param, val1, val2])

        if random_table:
            df_random = pd.DataFrame(random_table, columns = ['Parameter', 'Coef.', 'Std.Err.'])
            print("\n=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===")
            display(df_random)
        else:
            print("\nNo random-effects data found.")


In [9]:
# columns: item_id (indicates sentence set), wh_licensor (0/1), gap (0/1), island_type, surprisal

warnings.simplefilter("ignore", ConvergenceWarning)

def mixed_effects_linear_regression(df, iv_list, surprisal, label):
    '''
    Fits mixed-effects model and extracts wh-licensing interaction.
    '''
        
    for field in iv_list:
        if field == 'wh_numeric' or 'gap_numeric' or 'gap_distance_length':
            pass
        else:
            df[field] = df[field].astype('category')

    interaction_terms = ' * '.join(iv_list)
    random_effects = ' + '.join(iv_list)

    model = smf.mixedlm(
        f"{surprisal} ~ {interaction_terms}",
        df,
        groups = df["item_id"],
        #re_formula = f"~{random_effects}"
        re_formula = "1"
        )

    result = model.fit()
    #interaction_coef = result.params.get('wh_licensor[T.1]:gap[T.1]', None)

    print(f"\n=== {label.upper()} ===")
    #print(result.summary())
    print_summary(result)

    return result

#interaction = mixed_effects_linear_regression(df, "construction_type") # label name to be changed according to construction type

#### Gap Distance - Categorical/Continuous

In [10]:
sentence_df = pd.read_csv('test_sentences/Gap Distance.csv')
sentence_df['gap_distance'] = sentence_df['gap_distance'].fillna('null')
#sentence_df['gap_distance'].dropna(inplace = True)
#sentence_df = sentence_df[sentence_df['gap_distance'].replace(['nan', 'NaN'], np.nan).notna()]

In [11]:
sentence_df[['temp_mod', 'end']] = sentence_df['temp_mod'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'licensor', 'subj', 'modifier', 'verb', 'object', 'prep', 'goal', 'temp_mod']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['gap_distance_length'] = sentence_df['modifier'].apply(lambda x: len(x) if pd.notna(x) else 0)
sentence_df['grammaticality'] = sentence_df.apply(assign_grammaticality, axis = 1)

Gap Position == Object

In [12]:
# modifier: short_mod, med_mod, long_mod

In [13]:
object_df = sentence_df.copy()
#object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['prep', 'goal']), axis = 1)
object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['prep', 'goal'], 'object'), axis = 1)

# filter for object gap
objectgap_df = object_df[object_df['gap_position'] == 'DO']

In [14]:
# flip test: gap-
no_gap_df = objectgap_df[objectgap_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'region_surprisal', 'object gap position: gap-')


=== OBJECT GAP POSITION: GAP- ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: region_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 53.4851
Min. group size: 8
Log-Likelihood: -573.8300
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,21.424,2.832,7.564,0.000,15.873,26.975
1,wh_numeric,0.055,0.578,0.096,0.924,-1.078,1.188



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,153.738,7.588


In [15]:
# flip test: gap+
gap_df = objectgap_df[objectgap_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'region_surprisal', 'object gap position: gap+')


=== OBJECT GAP POSITION: GAP+ ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: region_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 110.5267
Min. group size: 8
Log-Likelihood: -619.3239
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,26.164,2.182,11.990,0.000,21.887,30.442
1,wh_numeric,0.130,0.831,0.156,0.876,-1.499,1.759



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,81.430,3.134


In [16]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(objectgap_df, iv_list, 'region_surprisal', 'object gap position: division by grammaticality')


=== OBJECT GAP POSITION: DIVISION BY GRAMMATICALITY ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 139.0278
Min. group size: 16
Log-Likelihood: -1261.0006
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,23.831,2.009,11.864,0.000,19.894,27.769
1,grammaticality[T.gram-],-0.075,1.318,-0.057,0.955,-2.658,2.509



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,63.321,2.043


Gap Postion == PP/Goal

In [17]:
pp_df = sentence_df.copy()
#pp_df['region_surprisal'] = pp_df.apply(lambda x: sum_region_surprisal(x, ['temp_mod']), axis = 1)
pp_df['region_surprisal'] = pp_df.apply(lambda x: sum_region_surprisal(x, ['temp_mod'], 'goal'), axis = 1)

# filter for PP gap
ppgap_df = pp_df[pp_df['gap_position'] == 'IO']

In [18]:
# flip test: gap-
no_gap_df = ppgap_df[ppgap_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'region_surprisal', 'pp gap position: gap-')


=== PP GAP POSITION: GAP- ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: region_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 68.0892
Min. group size: 8
Log-Likelihood: -583.8320
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,19.866,1.983,10.020,0.000,15.980,23.752
1,wh_numeric,0.038,0.652,0.058,0.954,-1.241,1.317



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,70.100,3.295


In [19]:
# flip test: gap+
gap_df = ppgap_df[ppgap_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'region_surprisal', 'pp gap position: gap+')


=== PP GAP POSITION: GAP+ ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: region_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 108.1907
Min. group size: 8
Log-Likelihood: -617.3687
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,22.070,2.129,10.367,0.000,17.897,26.243
1,wh_numeric,-0.274,0.822,-0.334,0.739,-1.886,1.337



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,77.119,3.014


In [20]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(ppgap_df, iv_list, 'region_surprisal', 'pp gap position: division by grammaticality')


=== PP GAP POSITION: DIVISION BY GRAMMATICALITY ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 132.3628
Min. group size: 16
Log-Likelihood: -1247.1581
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,20.812,1.493,13.935,0.000,17.885,23.739
1,grammaticality[T.gram-],0.312,1.286,0.243,0.808,-2.209,2.834



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,28.063,1.057


Embedded Clause

In [21]:
embed = ['subj', 'modifier', 'verb', 'object', 'prep', 'goal', 'temp_mod', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

Embedded Clause - Object Gap Position

In [22]:
# filter for object gap
embed_objectgap_df = sentence_df[sentence_df['gap_position'] == 'DO']

In [23]:
# flip test: gap-
no_gap_df = embed_objectgap_df[embed_objectgap_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'embed_surprisal', 'object gap position: gap-')


=== OBJECT GAP POSITION: GAP- ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.6671
Min. group size: 8
Log-Likelihood: -216.6043
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.878,0.178,44.153,0.000,7.529,8.228
1,wh_numeric,0.022,0.065,0.340,0.734,-0.105,0.149



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.553,0.270


In [24]:
# flip test: gap+
gap_df = embed_objectgap_df[embed_objectgap_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'embed_surprisal', 'object gap position: gap+')


=== OBJECT GAP POSITION: GAP+ ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.8648
Min. group size: 8
Log-Likelihood: -233.5445
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.650,0.168,45.415,0.000,7.320,7.980
1,wh_numeric,0.061,0.074,0.829,0.407,-0.083,0.205



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.459,0.211


In [25]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(embed_objectgap_df, iv_list, 'embed_surprisal', 'object gap position: division by grammaticality')


=== OBJECT GAP POSITION: DIVISION BY GRAMMATICALITY ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.8056
Min. group size: 16
Log-Likelihood: -444.2156
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.784,0.170,45.862,0.000,7.451,8.116
1,grammaticality[T.gram-],-0.039,0.100,-0.389,0.698,-0.236,0.158



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.475,0.196


Embedded Clause - PP/Goal Gap Position

In [26]:
# filter for PP gap
embed_ppgap_df = sentence_df[sentence_df['gap_position'] == 'IO']

In [27]:
# flip test: gap-
no_gap_df = embed_ppgap_df[embed_ppgap_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'embed_surprisal', 'pp gap position: gap-')


=== PP GAP POSITION: GAP- ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.6802
Min. group size: 8
Log-Likelihood: -217.9608
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.813,0.178,43.773,0.000,7.463,8.163
1,wh_numeric,-0.043,0.065,-0.662,0.508,-0.171,0.085



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.552,0.267


In [28]:
# flip test: gap+
gap_df = embed_ppgap_df[embed_ppgap_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'embed_surprisal', 'pp gap position: gap+')


=== PP GAP POSITION: GAP+ ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 1.0302
Min. group size: 8
Log-Likelihood: -248.1182
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.451,0.191,38.966,0.000,7.077,7.826
1,wh_numeric,-0.062,0.080,-0.779,0.436,-0.220,0.095



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.603,0.249


In [29]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(embed_ppgap_df, iv_list, 'embed_surprisal', 'pp gap position: division by grammaticality')


=== PP GAP POSITION: DIVISION BY GRAMMATICALITY ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 320
Method: REML
No. Groups: 20
Scale: 0.8793
Min. group size: 16
Log-Likelihood: -459.1543
Max. group size: 16
Converged: Yes
Mean group size: 16.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,7.623,0.186,40.927,0.000,7.258,7.988
1,grammaticality[T.gram-],0.019,0.105,0.184,0.854,-0.186,0.225



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.584,0.228


#### Double Gaps

In [30]:
sentence_df = pd.read_csv('test_sentences/Double Gaps.csv')

In [31]:
def assign_grammaticality_double_gaps(row):
    ''' Determines grammaticality based on licensor and gap values for the double gap construction.'''
    if row['filler'] == 1 and row['gap_count'] == 1:
        return 'gram+'
    elif row['filler'] == 0 and row['gap_count'] == 0:
        return 'gram+'
    elif row['filler'] == 1 and row['gap_count'] == 0:
        return 'gram-'
    elif row['filler'] == 0 and row['gap_count'] == 1:
        return 'gram-'
    elif row['filler'] == 0 and row['gap_count'] == 2:
        return 'gram-'
    elif row['filler'] == 1 and row['gap_count'] == 2:
        return 'gram-'

In [32]:
sentence_df[['modifier', 'end']] = sentence_df['modifier'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'licensor', 'subj', 'verb', 'object', 'modifier']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['gap_count'] = sentence_df['subject_gap'] + sentence_df['object_gap']
sentence_df['grammaticality'] = sentence_df.apply(assign_grammaticality_double_gaps, axis = 1)

Embedded Clause

In [33]:
embed = ['subj', 'object', 'verb', 'modifier', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [34]:
# flip test: gap = 0
gap_0 = sentence_df[sentence_df['gap_count'] == 0]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_0, iv_list, 'embed_surprisal', 'double gap embed: gap = 0')


=== DOUBLE GAP EMBED: GAP = 0 ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 40
Method: REML
No. Groups: 20
Scale: 0.0204
Min. group size: 2
Log-Likelihood: -24.1442
Max. group size: 2
Converged: Yes
Mean group size: 2.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.507,0.190,34.221,0.000,6.134,6.879
1,wh_numeric,0.076,0.023,3.344,0.001,0.031,0.120



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.713,2.322


In [35]:
# flip test: gap = 1
gap_1 = sentence_df[sentence_df['gap_count'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_1, iv_list, 'embed_surprisal', 'double gap embed: gap = 1')


=== DOUBLE GAP EMBED: GAP = 1 ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 80
Method: REML
No. Groups: 20
Scale: 0.3366
Min. group size: 4
Log-Likelihood: -90.6963
Max. group size: 4
Converged: Yes
Mean group size: 4.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.014,0.168,35.757,0.000,5.684,6.343
1,wh_numeric,0.085,0.065,1.310,0.190,-0.042,0.212



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.482,0.364


In [36]:
# flip test: gap = 2
gap_2 = sentence_df[sentence_df['gap_count'] == 2]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_2, iv_list, 'embed_surprisal', 'double gap embed: gap = 2')


=== DOUBLE GAP EMBED: GAP = 2 ===

=== MODEL SUMMARY ===
:            MixedLM Dependent Variable: embed_surprisal
No. Observations: 40
Method: REML
No. Groups: 20
Scale: 0.0195
Min. group size: 2
Log-Likelihood: -24.5721
Max. group size: 2
Converged: Yes
Mean group size: 2.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,5.230,0.199,26.304,0.000,4.840,5.619
1,wh_numeric,0.064,0.022,2.917,0.004,0.021,0.108



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.781,2.595


In [37]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'double gap embed: division by grammaticality')


=== DOUBLE GAP EMBED: DIVISION BY GRAMMATICALITY ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 160
Method: REML
No. Groups: 20
Scale: 0.4557
Min. group size: 8
Log-Likelihood: -188.1820
Max. group size: 8
Converged: Yes
Mean group size: 8.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,6.209,0.181,34.282,0.000,5.854,6.564
1,grammaticality[T.gram-],-0.430,0.110,-3.898,0.000,-0.646,-0.214



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,0.504,0.288


#### Wh-Islands

In [38]:
sentence_df = pd.read_csv('test_sentences/Wh-Islands.csv')
sentence_df['comp_type'] = sentence_df['comp_type'].fillna('null')

In [39]:
sentence_df[['cont', 'end']] = sentence_df['cont'].apply(split_ends)
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'filler_word', 'subj1', 'verb1', 'comp', 'subj2', 'embed_verb', 'obj', 'cont']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence (x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['comp_type'] = pd.Categorical(sentence_df['comp_type'], categories = ['null', 'that', 'whether'], ordered = True)
sentence_df['grammaticality'] = sentence_df.apply(assign_grammaticality, axis = 1)

Post-Gap Region - Continuation

In [40]:
sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['cont'], 'obj'), axis = 1)

In [41]:
# flip test: gap-
no_gap_df = sentence_df[sentence_df['gap_numeric'] == -1]
iv_list = ['wh_numeric', 'comp_type']
mixed_effects_linear_regression(no_gap_df, iv_list, 'region_surprisal', 'wh-islands continuation: gap-')


=== WH-ISLANDS CONTINUATION: GAP- ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.1387
Min. group size: 6
Log-Likelihood: -127.6484
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,19.297,1.182,16.321,0.000,16.980,21.615
1,comp_type[T.that],0.129,0.083,1.548,0.122,-0.034,0.292
2,comp_type[T.whether],0.394,0.083,4.733,0.000,0.231,0.557
3,wh_numeric,-0.075,0.059,-1.279,0.201,-0.191,0.040
4,wh_numeric:comp_type[T.that],0.144,0.083,1.727,0.084,-0.019,0.307
5,wh_numeric:comp_type[T.whether],0.202,0.083,2.430,0.015,0.039,0.366



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,27.890,26.634


In [42]:
# flip test: gap+
gap_df = sentence_df[sentence_df['gap_numeric'] == 1]
iv_list = ['wh_numeric', 'comp_type']
mixed_effects_linear_regression(gap_df, iv_list, 'region_surprisal', 'wh-islands continuation: gap+')


=== WH-ISLANDS CONTINUATION: GAP+ ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.3589
Min. group size: 6
Log-Likelihood: -183.4535
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,29.909,2.071,14.440,0.000,25.849,33.968
1,comp_type[T.that],0.224,0.134,1.671,0.095,-0.039,0.486
2,comp_type[T.whether],0.144,0.134,1.078,0.281,-0.118,0.407
3,wh_numeric,-0.015,0.095,-0.162,0.871,-0.201,0.170
4,wh_numeric:comp_type[T.that],0.166,0.134,1.240,0.215,-0.096,0.429
5,wh_numeric:comp_type[T.whether],0.176,0.134,1.313,0.189,-0.087,0.438



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,85.620,50.825


In [43]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(sentence_df, iv_list, 'region_surprisal', 'wh-islands continuation: division by grammaticality')


=== WH-ISLANDS CONTINUATION: DIVISION BY GRAMMATICALITY ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 59.4204
Min. group size: 12
Log-Likelihood: -845.4598
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,24.781,1.309,18.933,0.000,22.216,27.346
1,grammaticality[T.gram-],-0.059,0.995,-0.059,0.953,-2.009,1.892



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,24.359,1.286


Embedded Clause

In [44]:
embed = ['subj1', 'verb1', 'comp', 'subj2', 'embed_verb', 'obj', 'cont', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [45]:
# flip test: gap-
no_gap_df = sentence_df[sentence_df['gap_numeric'] == -1]
iv_list = ['wh_numeric', 'comp_type']
mixed_effects_linear_regression(no_gap_df, iv_list, 'embed_surprisal', 'wh-islands embed: gap-')


=== WH-ISLANDS EMBED: GAP- ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.0153
Min. group size: 6
Log-Likelihood: 1.3644
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.143,0.329,24.760,0.000,7.499,8.788
1,comp_type[T.that],-0.108,0.028,-3.915,0.000,-0.163,-0.054
2,comp_type[T.whether],0.076,0.028,2.748,0.006,0.022,0.130
3,wh_numeric,-0.214,0.020,-10.964,0.000,-0.253,-0.176
4,wh_numeric:comp_type[T.that],0.028,0.028,1.008,0.313,-0.026,0.082
5,wh_numeric:comp_type[T.whether],0.048,0.028,1.749,0.080,-0.006,0.103



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,2.156,6.196


In [46]:
# flip test: gap+
gap_df = sentence_df[sentence_df['gap_numeric'] == 1]
iv_list = ['wh_numeric', 'comp_type']
mixed_effects_linear_regression(gap_df, iv_list, 'embed_surprisal', 'wh-islands embed: gap+')


=== WH-ISLANDS EMBED: GAP+ ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.0217
Min. group size: 6
Log-Likelihood: -18.0604
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.122,0.382,21.246,0.000,7.373,8.872
1,comp_type[T.that],-0.148,0.033,-4.504,0.000,-0.213,-0.084
2,comp_type[T.whether],0.037,0.033,1.125,0.260,-0.027,0.102
3,wh_numeric,-0.242,0.023,-10.391,0.000,-0.288,-0.196
4,wh_numeric:comp_type[T.that],0.028,0.033,0.840,0.401,-0.037,0.092
5,wh_numeric:comp_type[T.whether],0.046,0.033,1.386,0.166,-0.019,0.110



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,2.912,7.033


In [47]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'wh-islands embed: division by grammaticality')


=== WH-ISLANDS EMBED: DIVISION BY GRAMMATICALITY ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 0.0938
Min. group size: 12
Log-Likelihood: -115.7095
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.095,0.355,22.793,0.000,7.399,8.791
1,grammaticality[T.gram-],0.029,0.040,0.721,0.471,-0.049,0.106



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,2.507,2.777


#### Adjunct Islands 

In [48]:
sentence_df = pd.read_csv('test_sentences/Adjunct Islands.csv')
sentence_df['adjunct_type'] = sentence_df['adjunct_type'].fillna('null')

In [49]:
sentence_df[['cont', 'end']] = sentence_df['cont'].apply(split_ends)
sentence_df['adjunct_front_trigger'] = np.nan
sentence_df['adjunct_front_trigger'] = sentence_df['adjunct_trigger'].where(sentence_df['adjunct_type'] == 'front', sentence_df['adjunct_front_trigger'])
sentence_df['adjunct_trigger'] = sentence_df['adjunct_trigger'].where(sentence_df['adjunct_type'] != 'front', np.nan)
sentence_df.rename(columns = {'adjunct_trigger': 'adjunct_back_trigger'}, inplace = True)
sentence_df = sentence_df[[col for col in sentence_df.columns if col != 'end'] + ['end']]

In [50]:
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
syntactic_parts = ['prefix', 'filler_word', 'adjunct_back_trigger', 'prep', 'subj', 'mod', 'verb', 'obj', 'cont', 'adjunct_front_trigger']
sentence_df['sentence'] = sentence_df.apply(lambda x: merge_sentence(x, syntactic_parts), axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['adjunct_type'] = pd.Categorical(sentence_df['adjunct_type'], categories = ['null', 'back', 'front'], ordered = True)
sentence_df['grammaticality'] = sentence_df.apply(assign_grammaticality, axis = 1)

Post-Gap Region - Continuation

In [51]:
sentence_df['region_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, ['cont'], 'obj'), axis = 1)

In [52]:
# flip test: gap-
no_gap_df = sentence_df[sentence_df['gap_numeric'] == -1]
iv_list = ['wh_numeric', 'adjunct_type']
mixed_effects_linear_regression(no_gap_df, iv_list, 'region_surprisal', 'adjunct islands continuation: gap-')


=== ADJUNCT ISLANDS CONTINUATION: GAP- ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.4036
Min. group size: 6
Log-Likelihood: -197.4706
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,31.021,3.228,9.611,0.000,24.695,37.347
1,adjunct_type[T.back],0.748,0.142,5.263,0.000,0.469,1.026
2,adjunct_type[T.front],-0.010,0.142,-0.068,0.946,-0.288,0.269
3,wh_numeric,0.326,0.100,3.248,0.001,0.129,0.523
4,wh_numeric:adjunct_type[T.back],-0.185,0.142,-1.302,0.193,-0.463,0.093
5,wh_numeric:adjunct_type[T.front],-0.436,0.142,-3.070,0.002,-0.715,-0.158



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,208.134,116.463


In [53]:
# flip test: gap+
gap_df = sentence_df[sentence_df['gap_numeric'] == 1]
iv_list = ['wh_numeric', 'adjunct_type']
mixed_effects_linear_regression(gap_df, iv_list, 'region_surprisal', 'adjunct islands continuation: gap+')


=== ADJUNCT ISLANDS CONTINUATION: GAP+ ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 1.0470
Min. group size: 6
Log-Likelihood: -243.1816
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,35.106,3.304,10.625,0.000,28.630,41.581
1,adjunct_type[T.back],1.288,0.229,5.629,0.000,0.840,1.736
2,adjunct_type[T.front],0.299,0.229,1.305,0.192,-0.150,0.747
3,wh_numeric,-0.037,0.162,-0.228,0.820,-0.354,0.280
4,wh_numeric:adjunct_type[T.back],-0.079,0.229,-0.346,0.729,-0.528,0.369
5,wh_numeric:adjunct_type[T.front],-0.336,0.229,-1.469,0.142,-0.785,0.112



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,217.796,75.704


In [54]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(sentence_df, iv_list, 'region_surprisal', 'adjunct islands continuation: division by grammaticality')


=== ADJUNCT ISLANDS CONTINUATION: DIVISION BY GRAMMATICALITY ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: region_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 118.6030
Min. group size: 12
Log-Likelihood: -933.2676
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,33.303,2.397,13.892,0.000,28.605,38.002
1,grammaticality[T.gram-],0.295,1.406,0.210,0.834,-2.461,3.050



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,95.176,3.263


Embedded Clause

In [55]:
embed = ['adjunct_back_trigger', 'prep', 'subj', 'mod', 'verb', 'obj', 'cont', 'adjunct_front_trigger', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)

In [56]:
# flip test: gap-
no_gap_df = sentence_df[sentence_df['gap_numeric'] == -1]
iv_list = ['wh_numeric', 'adjunct_type']
mixed_effects_linear_regression(no_gap_df, iv_list, 'embed_surprisal', 'adjunct islands embed: gap-')


=== ADJUNCT ISLANDS EMBED: GAP- ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.1098
Min. group size: 6
Log-Likelihood: -93.0924
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.701,0.347,25.084,0.000,8.021,9.381
1,adjunct_type[T.back],0.555,0.074,7.490,0.000,0.410,0.700
2,adjunct_type[T.front],0.310,0.074,4.185,0.000,0.165,0.455
3,wh_numeric,0.125,0.052,2.378,0.017,0.022,0.227
4,wh_numeric:adjunct_type[T.back],-0.097,0.074,-1.313,0.189,-0.242,0.048
5,wh_numeric:adjunct_type[T.front],-0.181,0.074,-2.447,0.014,-0.326,-0.036



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,2.352,2.542


In [57]:
# flip test: gap+
gap_df = sentence_df[sentence_df['gap_numeric'] == 1]
iv_list = ['wh_numeric', 'adjunct_type']
mixed_effects_linear_regression(gap_df, iv_list, 'embed_surprisal', 'adjunct islands embed: gap+')


=== ADJUNCT ISLANDS EMBED: GAP+ ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 120
Method: REML
No. Groups: 20
Scale: 0.1545
Min. group size: 6
Log-Likelihood: -109.7256
Max. group size: 6
Converged: Yes
Mean group size: 6.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.098,0.355,22.827,0.000,7.403,8.793
1,adjunct_type[T.back],0.742,0.088,8.436,0.000,0.569,0.914
2,adjunct_type[T.front],0.502,0.088,5.716,0.000,0.330,0.675
3,wh_numeric,0.152,0.062,2.443,0.015,0.030,0.274
4,wh_numeric:adjunct_type[T.back],-0.125,0.088,-1.426,0.154,-0.298,0.047
5,wh_numeric:adjunct_type[T.front],-0.208,0.088,-2.371,0.018,-0.381,-0.036



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,2.440,2.229


In [58]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(sentence_df, iv_list, 'embed_surprisal', 'adjunct islands embed: division by grammaticality')


=== ADJUNCT ISLANDS EMBED: DIVISION BY GRAMMATICALITY ===

=== MODEL SUMMARY ===
Model: MixedLM
Dependent Variable: embed_surprisal
No. Observations: 240
Method: REML
No. Groups: 20
Scale: 0.3984
Min. group size: 12
Log-Likelihood: -273.1928
Max. group size: 12
Converged: Yes
Mean group size: 12.0

=== FIXED EFFECTS COEFFICIENTS ===


,Parameter,Coef.,Std.Err.,z,P>|z|,[0.025,0.975]
0,Intercept,8.755,0.341,25.688,0.000,8.087,9.423
1,grammaticality[T.gram-],-0.009,0.081,-0.109,0.913,-0.169,0.151



=== RANDOM EFFECTS / VARIANCE-COVARIANCE ===


,Parameter,Coef.,Std.Err.
0,Group Var,2.257,1.227


#### Complex NP Islands

In [ ]:
sentence_df = pd.read_csv('test_sentences/Complex NP Islands.csv')

In [ ]:
sentence_df = encode_wh_licensor(sentence_df)
sentence_df['gap_numeric'] = sentence_df['gap'].replace(0, -1)
sentence_df['sentence'] = sentence_df.apply(merge_sentence, axis = 1)
sentence_df['surprisals'] = sentence_df['sentence'].apply(calculate_surprisal)
sentence_df['grammaticality'] = sentence_df.apply(assign_grammaticality, axis = 1)

Subject Condition

In [ ]:
subject_df = sentence_df.copy()

In [ ]:
subject_df['region_surprisal'] = subject_df.apply(lambda x: sum_region_surprisal(x, ['subj_setup']), axis = 1)
subject_df

In [ ]:
# filter for subject condition
subject_df = subject_df[subject_df['subj_obj'] == 'subject']

In [ ]:
# flip test: gap-
no_gap_df = subject_df[subject_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'region_surprisal', 'complex np islands subject condition: gap-')

In [ ]:
# flip test: gap+
gap_df = subject_df[subject_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'region_surprisal', 'complex np islands subject condition: gap+')

In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(subject_df, iv_list, 'region_surprisal', 'complex np islands subject condition: division by grammaticality')

Object Condition

In [ ]:
object_df = sentence_df.copy()

In [ ]:
object_df['region_surprisal'] = object_df.apply(lambda x: sum_region_surprisal(x, ['end']), axis = 1)
object_df

In [ ]:
# filter for object gap
object_df = object_df[object_df['subj_obj'] == 'object']

In [ ]:
# flip test: gap-
no_gap_df = object_df[object_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'region_surprisal', 'complex np islands object condition: gap-')

In [ ]:
# flip test: gap+
gap_df = object_df[object_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'region_surprisal', 'complex np islands object condition: gap+')

In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(object_df, iv_list, 'region_surprisal', 'complex np islands object condition: division by grammaticality')

Embedded Clause

In [ ]:
embed = ['subj', 'that_rc', 'what_rc', 'rc_np', 'prep', 'prep_np', 'subj_setup', 'obj_setup', 'end']
sentence_df['embed_surprisal'] = sentence_df.apply(lambda x: sum_region_surprisal(x, embed, normalize = True), axis = 1)
sentence_df

Embedded Clause - Subject Condition

In [ ]:
# filter for subject condition
subject_df = sentence_df[sentence_df['subj_obj'] == 'subject']

In [ ]:
# flip test: gap-
no_gap_df = subject_df[subject_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'embed_surprisal', 'complex np islands subject condition: gap-')

In [ ]:
# flip test: gap+
gap_df = subject_df[subject_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'embed_surprisal', 'complex np islands subject condition: gap+')

In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(subject_df, iv_list, 'embed_surprisal', 'complex np islands subject condition: division by grammaticality')

Embedded Clause - Object Condition

In [ ]:
# filter for object gap
object_df = sentence_df[sentence_df['subj_obj'] == 'object']

In [ ]:
# flip test: gap-
no_gap_df = object_df[object_df['gap_numeric'] == -1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(no_gap_df, iv_list, 'embed_surprisal', 'complex np islands object condition: gap-')

In [ ]:
# flip test: gap+
gap_df = object_df[object_df['gap_numeric'] == 1]
iv_list = ['wh_numeric']
mixed_effects_linear_regression(gap_df, iv_list, 'embed_surprisal', 'complex np islands object condition: gap+')

In [ ]:
# division by grammaticality
iv_list = ['grammaticality']
mixed_effects_linear_regression(object_df, iv_list, 'embed_surprisal', 'complex np islands object condition: division by grammaticality')